<a href="https://colab.research.google.com/github/victorvieirat/yardsalemodel/blob/main/yardsale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:

#@title Configurações

import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm

import pandas as pd
import plotly.express as px
import pandas._testing as tm

import plotly.graph_objects as go

imps_riq_ricos = .05
imps_riq_pobres = .01
imps_riq_resto = .02

imps_lucro = 0.1

args = {'font':dict(family="Arial",
                    size=18),
        'legend':dict(font=dict(size=18),orientation="h", yanchor="top", xanchor="left", y=0.98,x=0.02),
        'margin':dict(l=5, r=5, t=50, b=5),
        'width':700,
        'height':500
        }

def unpivot(frame):
    N, K = frame.shape
    data = {
        "Riqueza": frame.to_numpy().ravel("F"),
        "pessoa": np.asarray(frame.columns).repeat(N),
        "iteracao": np.tile(np.asarray(frame.index), K),
    }
    return pd.DataFrame(data, columns=["iteracao", "pessoa", "Riqueza"])

def multiplicar(i):
  if i >=round((1-raz_ricos)*num_populacao):
    return (1-imps_riq_ricos)
  elif i >=round(raz_pobres*num_populacao):
    return (1-imps_riq_pobres)
  else:
    return (1-imps_riq_resto)

In [46]:
#@title Parâmetros do usuário
tem_imposto = False
def convert_bool(input):
  if input == 'Sem':
    return False
  else:
    return True
#@markdown Tamanho da população
num_populacao = 100 #@param {type:"slider", min:10, max:500, step:10e0}
#@markdown Quantidade população
iteracoes = 30000 #@param {type:"slider", min:1000, max:15e4, step:10e2}
#@markdown Razão dos ricos
raz_ricos = 0.05 #@param {type:"slider", min:0.01, max:0.5, step:0.01}
#@markdown Razão dos pobres
raz_pobres = 0.5 #@param {type:"slider", min:0.01, max:0.9, step:0.01}
#@markdown Tributação na lucro 
tax_lucro = 'Sem' #@param ["Sem", "Com"]
#@markdown Tributação na riqueza
tax_riqueza = 'Com' #@param ["Sem", "Com"]
tax_riqueza_tipo = 'Fixa' #@param ["Sem","Fixa", "Progressiva"]
#@markdown Distribuição da riqueza
distribuicao = 'Fixa' #@param ["Fixa", "Regressiva"]

print(f'População atual: {num_populacao}')
print(f'Quantidade de iterações atual: {iteracoes}')
print(f'Você irá visualizar os {round(raz_ricos*100)}% mais ricos e {round(raz_pobres*100)}% mais pobres.')
if convert_bool(tax_lucro):
  tem_imposto=True
  print(f'Com taxação de {round(imps_lucro*100)}% sobre o lucro')
else:
  print(f'Sem taxação sobre o lucro')
if convert_bool(tax_riqueza):
  tem_imposto=True
  if(tax_riqueza_tipo == 'Fixa'):
    print(f'Com taxa fixa  de {round(imps_riq_resto*100)}% para todos')
  else:
    print(f'Com taxa de {round(imps_riq_ricos*100)}% para os {round(raz_ricos*100,2)}% mais ricos')
    print(f'Com taxa de {round(imps_riq_pobres*100)}% para os {round(raz_pobres*100,2)}% mais pobres')
    print(f'Com taxa de {round(imps_riq_resto*100)}% para os demais')
else:
  print(f'Sem taxação sobre riqueza')
if tem_imposto:
  if distribuicao == 'Fixa':
      print(f'Com redistribuição dos impostos igual para todos')
  else:
    print(f'Com redistribuição dos impostos inversamente proporcional à renda')


População atual: 100
Quantidade de iterações atual: 30000
Você irá visualizar os 5% mais ricos e 50% mais pobres.
Sem taxação sobre o lucro
Com taxa fixa  de 2% para todos
Com redistribuição dos impostos igual para todos


In [47]:
#@title Simulação
populacao = [100 for i in range(0,num_populacao)]
total = sum(populacao)
pobres =[]
ricos = []
time_series = []
max =0
num = 0
qnt_porcentagens = 1000
qnt_frames = iteracoes if iteracoes < 50 else 50
qnt_porcentagens = iteracoes if iteracoes < 1000 else 1000

for it in tqdm(range(0,iteracoes)):
  # índice das pessoas que vão se encontrar
  indices_elegiveis = [i for i, x in enumerate(np.array(populacao) > 1) if x ]
  if len(indices_elegiveis) >=2:
    A,B = random.sample(indices_elegiveis,2)
    #O valor é proposto pelo que tem menos renda
    if populacao[A] > populacao[B]:
      valor = populacao[B]*0.1
    else:
      valor = populacao[A]*0.1
    if (convert_bool(tax_lucro)):
      valor *= (1-imps_lucro)
    #Aleatoriamente um ganha e outro perder
    ganha,perde = random.sample([A,B],2)
    populacao[perde] -= round(valor,3)
    if convert_bool(tax_lucro):
      valor *= 0.9
    populacao[ganha] += round(valor,3)
  if tem_imposto and (((num) % num_populacao == 0) or it == (iteracoes -1)):
    if convert_bool(tax_riqueza):
      if(tax_riqueza_tipo == 'Fixa'):
        populacao = [i*(1-imps_riq_resto) for i in populacao]
      else:
        for i,x in enumerate(populacao):
          populacao[i]*= multiplicar(i)
      diff = (100.0*num_populacao) - np.round(sum(populacao),3)
      impostos = diff if diff > 0 else 0
      if impostos != 0:
        if distribuicao == 'Fixa':
          populacao = [x+(impostos/len(populacao)) for x in populacao]
        if distribuicao == 'Regressiva':
          temp = [(1/i) for i in [(i/sum(populacao)) for i in populacao]]
          populacao = np.add([(i/np.sum(temp))*impostos for i in temp], populacao)
    
  populacao.sort()
  #Variável auxiliar nos gráficos
  if populacao[-1] > max:
    max = populacao[-1]
  populacao = np.round(populacao,3)
  # Para não ter problemas de memória essas variáveis são atualizadas
  # a cada determinado número de iterações
  if num % int(iteracoes/qnt_frames) == 0:
    time_series.append(np.round(populacao,2).copy())
  if num % int(iteracoes/qnt_porcentagens) == 0:
    pobres.append((sum(populacao[:round(num_populacao*raz_pobres)])/total)*100)
    ricos.append((sum(populacao[round(num_populacao*(1-raz_ricos)):])/total)*100)
  num+=1

100%|██████████| 30000/30000 [00:01<00:00, 16062.49it/s]


In [48]:
fig = go.Figure()
x = [i for i in range(0,iteracoes,int(iteracoes/qnt_porcentagens))]
fig.add_trace(go.Scatter(x=x, y=pobres,
                    mode='lines',
                    name=f'{round(raz_pobres*100)}% mais pobres'))
fig.add_trace(go.Scatter(x=x, y=ricos,
                    mode='lines',
                    name=f'{round(raz_ricos*100)}% mais ricos'))

fig.update_layout(**args, title="Riqueza fixa e redisribuição fixa",yaxis_ticksuffix = "%")
fig.show()

In [29]:
df = unpivot(pd.DataFrame(time_series))
df['iteracao'] = (df['iteracao']*(iteracoes/qnt_frames)).astype(int)
df['Percentual'] = round((df['Riqueza']/total)*100,2).astype(str) + '%'
fig = px.bar(df, 
             x='pessoa',
             y="Riqueza",
             color="Riqueza",
             color_continuous_scale='Turbo',
             range_color=[0,total*0.1],
             #color_continuous_midpoint=total*0.1,
             
              hover_data={'iteracao':False,
                          'pessoa':False,
                          'Riqueza':False,
                          'Percentual':True},
             animation_frame="iteracao",
             animation_group='pessoa',
             )

fig.update_xaxes(showticklabels=False)

#fig.update_yaxes(showticklabels=False)
fig.update_layout(**args,
                  title="Riqueza ordenadas ao longo do tempo",
                  xaxis_title="Pessoa",
                  yaxis_title="Quantidade de riqueza"
                  )
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 100
fig.update_layout(hoverlabel=dict(font_size=16),
                  coloraxis_colorbar=dict(thickness=20,
                                          title='% do total',
                                          tickvals=[round(i*(num_populacao*100)) for i in [0.05,0.1,0.2,0.3,0.4]],
                                          ticktext=["5%","10%","20%","30%","40%"]
                                          )
                  )

fig.layout.coloraxis.colorbar.title.font = dict(size=14)
fig.layout.coloraxis.colorbar.tickfont = dict(size=14)
fig.show()